In [4]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI()

a = chat.predict("How many planets are there?")  # 모델 객체.predict("질문 내용")을 통해 모델에 질문을 함.

a 
# 'As of now, there are officially eight planets in our solar system. These planets are Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. However, it is important to note that there is ongoing debate and discussion regarding the status of Pluto as a planet. Pluto was reclassified as a "dwarf planet" by the International Astronomical Union (IAU) in 2006.'

'As of now, there are officially eight planets in our solar system. These planets are Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. However, it is important to note that there is ongoing debate and discussion regarding the status of Pluto as a planet. Pluto was reclassified as a "dwarf planet" by the International Astronomical Union (IAU) in 2006.'

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage

chat = ChatOpenAI(temperature=0.1) # temperature속성을 통해 모델의 창의력을 설정해줌.

messages = [ # 메세지 리스트
    SystemMessage(content="You are geography expert. And you only reply in Korean."), # SystemMessage(content="모델 설정 내용")
    AIMessage(content="Hello, My Name is Paul!"), # AIMessage(content="시작 인사말 내용")
    HumanMessage(content="What is the distance between Mexico and Thailand?"), # HumanMessage(content="실제 질문 내용")
]

a = chat.predict_messages(messages) # 모델 객체.predict_messages(메세지 리스트)를 통해 모델에 질문을 함. 

a
# AIMessage(content='안녕하세요, 제 이름은 폴이에요! 멕시코와 태국 사이의 거리는 대략 17,000 킬로미터입니다.')

AIMessage(content='안녕하세요! 저는 지리 전문가입니다. 멕시코와 태국 사이의 거리는 대략 17,000 킬로미터입니다. 제 이름은 폴이에요! 어떻게 도와드릴까요?')

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

chat = ChatOpenAI(temperature=0.1) 

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}",
) # PromptTemplate.from_template("질문 내용 {변수명}")을 통해 템플릿 객체를 생성함.

prompt = template.format(country_a="Mexico", country_b="Thailand") # 템플릿 객체.format(변수명="변수값")를 통해 프롬프트를 생성함.


a = chat.predict(prompt) # 모델 객체.predict(프롬프트)를 통해 모델에 질문을 함.

a
# 'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles).'


'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles).'

In [19]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1) 

template = ChatPromptTemplate.from_messages( # ChatPromptTemplate.from_messages([ 메세지 튜플{변수명} ])을 통해 템플릿 객체를 생성함.
    [ 
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "Hello, my name is {name}!"),
        ("human", "What is the distance between {country_a} and {country_b}"),
    ]
)

ChatPrompt = template.format_messages( # 템플릿 객체.format_messages(변수명="변수값")을 통해 챗 프롬프트를 생성함.
    language="Korean",
    name="Paul",
    country_a="Mexico",
    country_b="Thailand"
)

a = chat.predict_messages(ChatPrompt) # 모델 객체.predict_messages(챗 프롬프트)를 통해 모델에 질문을 함.

a
# AIMessage(content='안녕하세요, 제 이름은 폴이에요! 멕시코와 태국 사이의 거리는 대략 17,000 킬로미터입니다.')

AIMessage(content='멕시코와 태국 사이의 거리는 약 17,000 킬로미터입니다.')

In [ ]:
############################################

In [3]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items =  text.strip().split(",")
        return list(map(str.strip, items)) 


p = CommaOutputParser()
p.parse("Hello, how, are, you")

# ['Hello', 'how', 'are', 'you']

['Hello', 'how', 'are', 'you']

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate


chat = ChatOpenAI(temperature=0.1) 


template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do not reply with anything else."),
    ("human", "{question}")
])

prompt = template.format_messages(max_items=10, question="What are the colors?")

result = chat.predict_messages(prompt) 
result  # AIMessage(content='red, orange, yellow, green, blue, indigo, violet, black, white, gray')

p = CommaOutputParser()

parsedResult = p.parse(result.content)

parsedResult

# ['red',
#  'orange',
#  'yellow',
#  'green',
#  'blue',
#  'indigo',
#  'violet',
#  'black',
#  'white',
#  'gray']

['red',
 'orange',
 'yellow',
 'green',
 'blue',
 'indigo',
 'violet',
 'black',
 'white',
 'gray']

In [11]:
chain = template | chat | CommaOutputParser()

chain.invoke ({
    "max_items": 10,
    "question": "What are the colors?"
})

# ['red',
#  'orange',
#  'yellow',
#  'green',
#  'blue',
#  'indigo',
#  'violet',
#  'black',
#  'white',
#  'gray']

['red',
 'orange',
 'yellow',
 'green',
 'blue',
 'indigo',
 'violet',
 'black',
 'white',
 'gray']

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]) 


chef_template = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food.")
])


chef_chain = chef_template | chat


In [ ]:
veg_template = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}")
])

veg_chain = veg_template | chat

final_chain = {"recipe":chef_chain} | veg_chain

final_chain.invoke({
    "cuisine": "Korean"
})


"""
Great choice! Korean cuisine is known for its bold flavors and unique combinations. Here's a recipe for a classic Korean dish called Bibimbap:

Ingredients:
- 2 cups cooked rice
- 1 carrot, julienned
- 1 zucchini, julienned
- 1 cup spinach
- 1 cup bean sprouts
- 4 shiitake mushrooms, sliced
- 1/2 pound beef (can be substituted with tofu or chicken), thinly sliced
- 4 eggs
- 2 tablespoons vegetable oil
- 2 tablespoons soy sauce
- 1 tablespoon sesame oil
- 1 tablespoon gochujang (Korean chili paste)
- Salt, to taste
- Sesame seeds, for garnish
- Kimchi, for serving (optional)

Instructions:
1. Cook the rice according to the package instructions and set aside.
2. Blanch the spinach and bean sprouts in boiling water for 1-2 minutes. Drain and rinse with cold water. Squeeze out any excess water from the spinach and season with a pinch of salt and 1 teaspoon of sesame oil. Set aside.
3. Heat 1 tablespoon of vegetable oil in a pan over medium heat. Stir-fry the carrots and zucchini for 2-3 minutes until slightly softened. Season with a pinch of salt and set aside.
4. In the same pan, add another tablespoon of vegetable oil and cook the mushrooms until they are tender. Season with soy sauce and set aside.
5. In a separate pan, cook the beef (or tofu/chicken) over medium-high heat until browned and cooked through. Season with soy sauce and set aside.
6. In the same pan, fry the eggs sunny-side up or to your desired doneness.
7. To assemble the bibimbap, divide the rice into four bowls. Arrange the cooked vegetables and beef (or tofu/chicken) on top of the rice.
8. In a small bowl, mix together gochujang, 1 tablespoon of soy sauce, and 1 tablespoon of sesame oil. Drizzle this sauce over the rice and vegetables.
9. Top each bowl with a fried egg and sprinkle with sesame seeds.
10. Serve the bibimbap with kimchi on the side, if desired.

Enjoy your homemade Korean Bibimbap!Great choice! Bibimbap is a delicious Korean dish that can easily be made vegetarian. Here's how you can make a vegetarian version of Bibimbap:

Ingredients:
- 2 cups cooked rice
- 1 carrot, julienned
- 1 zucchini, julienned
- 1 cup spinach
- 1 cup bean sprouts
- 4 shiitake mushrooms, sliced
- 1/2 pound tofu, thinly sliced
- 4 eggs (optional, omit for a vegan version)
- 2 tablespoons vegetable oil
- 2 tablespoons soy sauce
- 1 tablespoon sesame oil
- 1 tablespoon gochujang (Korean chili paste)
- Salt, to taste
- Sesame seeds, for garnish
- Kimchi, for serving (optional)

Instructions:
1. Cook the rice according to the package instructions and set aside.
2. Blanch the spinach and bean sprouts in boiling water for 1-2 minutes. Drain and rinse with cold water. Squeeze out any excess water from the spinach and season with a pinch of salt and 1 teaspoon of sesame oil. Set aside.
3. Heat 1 tablespoon of vegetable oil in a pan over medium heat. Stir-fry the carrots and zucchini for 2-3 minutes until slightly softened. Season with a pinch of salt and set aside.
4. In the same pan, add another tablespoon of vegetable oil and cook the mushrooms until they are tender. Season with soy sauce and set aside.
5. In a separate pan, cook the tofu over medium-high heat until browned and cooked through. Season with soy sauce and set aside.
6. In a separate pan, fry the eggs sunny-side up or to your desired doneness (omit for a vegan version).
7. To assemble the bibimbap, divide the rice into four bowls. Arrange the cooked vegetables and tofu on top of the rice.
8. In a small bowl, mix together gochujang, 1 tablespoon of soy sauce, and 1 tablespoon of sesame oil. Drizzle this sauce over the rice and vegetables.
9. Top each bowl with a fried egg (if using) and sprinkle with sesame seeds.
10. Serve the bibimbap with kimchi on the side, if desired.

Enjoy your homemade vegetarian Korean Bibimbap!

"""

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]) 

t = PromptTemplate(
    template="What is the capital of {country}",
    input_variables=["country"],
)

prompt = t.format(country="France")

chat.predict(prompt)

# The capital of France is Paris.


In [22]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]) 



examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_template = """
    Q: {question}
    A: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)


prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)


chain = prompt | chat

chain.invoke({
    "country": "Korea"
})

"""
Teacher: 
I know this:
Capital: Seoul
Language: Korean
Food: Kimchi and Bibimbap
Currency: South Korean Won
"""

Teacher: 
I know this:
Capital: Seoul
Language: Korean
Food: Kimchi and Bibimbap
Currency: South Korean Won

AIMessageChunk(content='Teacher: \nI know this:\nCapital: Seoul\nLanguage: Korean\nFood: Kimchi and Bibimbap\nCurrency: South Korean Won')

In [27]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate


chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]) 



examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {   "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]




example_prompt = ChatPromptTemplate.from_messages([
    ("Q", "what do you know about {country}?"),
    ("A", "{answer}")
])


example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
)


final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert, you give short answers."),
    example_prompt,
    ("human", "What do you know about {country}")
])


chain = final_prompt | chat

chain.invoke({
    "country": "USA"
})


"""
  I know this:
        Capital: Washington, D.C.
        Language: English
        Food: Hamburgers and Hotdogs
        Currency: United States Dollar (USD)
"""


        I know this:
        Capital: Washington, D.C.
        Language: English
        Food: Hamburgers and Hotdogs
        Currency: United States Dollar (USD)
        

AIMessageChunk(content='\n        I know this:\n        Capital: Washington, D.C.\n        Language: English\n        Food: Hamburgers and Hotdogs\n        Currency: United States Dollar (USD)\n        ')

In [39]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector


chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]) 


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]





example_template = """
    Q: {question}
    A: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)


example_selector = LengthBasedExampleSelector(
    examples = examples,
    example_prompt=example_prompt,
    max_length=100,
)


prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    example_selector = example_selector,
    suffix="Q: What do you know about {country}?",
    input_variables=["country"]
)


prompt.format(country="Brazil")

# '\n    Q: What do you know about France?\n    A: \n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\n\nQ: What do you know about Brazil?'


'\n    Q: What do you know about France?\n    A: \n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\n\nQ: What do you know about Brazil?'

In [37]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector


chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]) 


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]



class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]





example_template = """
    Q: {question}
    A: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)


example_selector = RandomExampleSelector(
    examples = examples,
)


prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    example_selector = example_selector,
    suffix="Q: What do you know about {country}?",
    input_variables=["country"]
)


prompt.format(country="Brazil")
# '\n    Q: What do you know about France?\n    A: \n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\n\nQ: What do you know about Brazil?'


'\n    Q: What do you know about Italy?\n    A: \n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\n\nQ: What do you know about Brazil?'

In [ ]:
############################################

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.json")


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

prompt.format(country="Germany")

# 'What is the capital of Germany'

'What is the capital of Germany'

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.yaml")


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

prompt.format(country="Germany")

# 'What is the capital of Germany'

'What is the capital of Germany'

In [13]:

from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]


full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)


chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

# Arrrrg! Me favorite food be a good ol' plate of rum-infused shrimp! The taste of those succulent crustaceans be like a party in me mouth, matey! Arg arg!


Arrrrg! Me favorite food be a good ol' plate of rum-infused shrimp! The taste of those succulent crustaceans be like a party in me mouth, matey! Arg arg!

AIMessageChunk(content="Arrrrg! Me favorite food be a good ol' plate of rum-infused shrimp! The taste of those succulent crustaceans be like a party in me mouth, matey! Arg arg!")

In [16]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())
set_debug(True)

chat = ChatOpenAI(
    temperature=0.1,
)

chat.predict("How do you make italian pasta")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [23.92s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork or your fingers, gradually mix the eggs and salt into the flour, incorporating a little at a time.\n4. Once the dough starts to come together, knead it with your hands until it forms a smooth and elastic ball. If the dough is too dry, you can add a little water, teaspoon by teaspoon, until it reaches the desired consistency. If it's too wet, add a little more flour.\n5. 

"To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork or your fingers, gradually mix the eggs and salt into the flour, incorporating a little at a time.\n4. Once the dough starts to come together, knead it with your hands until it forms a smooth and elastic ball. If the dough is too dry, you can add a little water, teaspoon by teaspoon, until it reaches the desired consistency. If it's too wet, add a little more flour.\n5. Once the dough is formed, cover it with a clean kitchen towel or plastic wrap and let it rest for about 30 minutes. This allows the gluten to relax and makes the dough easier to work with.\n6. After resting, divide the dough into smaller portio

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import SQLiteCache

set_llm_cache(SQLiteCache("cache.db"))

set_debug(True)

chat = ChatOpenAI(
    temperature=0.1,
)

chat.predict("How do you make italian pasta")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [22.94s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork or your fingers, gradually mix the eggs and salt into the flour, incorporating a little bit at a time.\n4. Once the dough starts to come together, knead it with your hands until it forms a smooth and elastic ball. If the dough is too dry, you can add a little water, one tablespoon at a time, until it reaches the desired consistency.\n5. Once the dough is formed, cover it

"To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork or your fingers, gradually mix the eggs and salt into the flour, incorporating a little bit at a time.\n4. Once the dough starts to come together, knead it with your hands until it forms a smooth and elastic ball. If the dough is too dry, you can add a little water, one tablespoon at a time, until it reaches the desired consistency.\n5. Once the dough is formed, cover it with a clean kitchen towel or plastic wrap and let it rest for about 30 minutes. This will allow the gluten to relax and make the dough easier to work with.\n6. After resting, divide the dough into smaller portions, depending on the amount of 

In [15]:

chat.predict("How do you make italian pasta")

"To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork or your fingers, gradually mix the eggs and salt into the flour, incorporating a little at a time.\n4. Once the dough starts to come together, knead it with your hands until it forms a smooth and elastic ball. If the dough is too dry, you can add a little water, teaspoon by teaspoon, until it reaches the desired consistency. If it's too wet, add a little more flour.\n5. Once the dough is formed, cover it with a clean kitchen towel or plastic wrap and let it rest for about 30 minutes. This will allow the gluten to relax and make the dough easier to work with.\n6. After resting, divide the dough into smaller por

In [19]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1
)

with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for soju")
    b = chat.predict("What is the recipe for bread")
    print(a, b, "\n")
    print(usage)


[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: What is the recipe for soju"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Here is a simple recipe for making soju at home:\n\nIngredients:\n- 1 cup of rice\n- 1 cup of nuruk (a Korean fermentation starter)\n- 8 cups of water\n- 1 tablespoon of yeast (optional, for faster fermentation)\n- Sugar (optional, for sweetness)\n\nInstructions:\n1. Rinse the rice thoroughly and soak it in water for about 1 hour.\n2. Drain the soaked rice and transfer it to a large pot. Add 8 cups of water and bring it to a boil.\n3. Reduce the heat to low and simmer the rice for about 30 minutes, or until it becomes soft and mushy.\n4. Remove the pot from heat and let it cool down to room temperature.\n5. Once the rice has cooled, transfer it to a large fermentation container or jar.\n6. Add the nuruk to the container and mix it well with the rice. If

In [20]:
from langchain.llms.openai import OpenAI

chat = OpenAI(
    temperature=0.1,
    max_tokens=450,
    model="gpt-3.5-turbo-16k"
)

chat.save("model.json")

In [21]:
from langchain.llms.loading import load_llm

chat = load_llm("model.json")

chat

/Users/gimseongjin/FullStack_GPT/env/lib/python3.11/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/gimseongjin/FullStack_GPT/env/lib/python3.11/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


OpenAIChat(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo-16k', model_kwargs={'temperature': 0.1, 'max_tokens': 450, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}})

In [24]:
from langchain.memory import ConversationBufferMemory # stores whole previous messages 

memory = ConversationBufferMemory(return_messages=True)

memory.save_context({"input": "Hi"}, {"output": "How are you?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi'), AIMessage(content='How are you?')]}

In [25]:

memory.save_context({"input": "Hi"}, {"output": "How are you?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi'),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi'),
  AIMessage(content='How are you?')]}

In [26]:

memory.save_context({"input": "Hi"}, {"output": "How are you?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi'),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi'),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi'),
  AIMessage(content='How are you?')]}

In [28]:
from langchain.memory import ConversationBufferWindowMemory # stores only recent previous message



memory = ConversationBufferWindowMemory(
    return_messages=True, 
    k=4 # how many store
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


add_message(1,1)



In [29]:
add_message(2, 2)
add_message(3, 3)
add_message(4, 4)

In [30]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='1'),
  AIMessage(content='1'),
  HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4')]}

In [32]:
add_message(5, 5)

In [33]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}

In [34]:

from langchain.memory import ConversationSummaryMemory 
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")


[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "summary": "",
  "new_lines": "Human: Hi I'm Nicolas, I live in South Korea\nAI: Wow that is so cool!"
}
[llm/start] [1:chain:LLMChain > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.\n\nEXAMPLE\nCurrent summary:\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.\n\nNew lines of conversation:\nHuman: Why do you think artificial intelligence is a force for good?\nAI: Because artificial intelligence will help humans reach their full potential.\n\nNew summary:\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.\nEND OF EXAMPLE\n\nCurrent summary:\n\n\nNew lines of conversation:\nHuman:

In [36]:
add_message("South Korea is so pretty", "I wish I could go!!!")


[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "summary": "The human introduces themselves as Nicolas and mentions that they live in South Korea. The AI responds with enthusiasm, saying that it thinks that is cool and expresses a desire to visit because it thinks South Korea is pretty.",
  "new_lines": "Human: South Kddorea is so pretty\nAI: I wish I could go!!!"
}
[llm/start] [1:chain:LLMChain > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.\n\nEXAMPLE\nCurrent summary:\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.\n\nNew lines of conversation:\nHuman: Why do you think artificial intelligence is a force for good?\nAI: Because artificial intelligence will help humans reach their full potential.\n\nNew summary:\nThe human asks what the AI thinks o

In [37]:
get_history()

{'history': 'The human introduces themselves as Nicolas and mentions that they live in South Korea. The AI responds with enthusiasm, saying that it thinks that is cool and expresses a desire to visit because it thinks South Korea is pretty. The human agrees and says that South Korea is indeed very beautiful, to which the AI responds by expressing its own wish to visit.'}

In [1]:
from langchain.memory import ConversationSummaryBufferMemory # ConversationSummaryMemory + ConversationBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=80, # If it exceeds the max_token_limit, oldest conversation will be summarized
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})



add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")

In [2]:
get_history()

{'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea"),
  AIMessage(content='Wow that is so cool!')]}

In [3]:
add_message("South Korea is so pretty", "I wish I could go!!!")


In [4]:
get_history()

{'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea"),
  AIMessage(content='Wow that is so cool!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!!')]}

In [5]:
add_message("How far is Korea from Argentina?", "I don`t know! Super far!")

In [6]:
get_history()

{'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea"),
  AIMessage(content='Wow that is so cool!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!!'),
  HumanMessage(content='How far is Korea from Argentina?'),
  AIMessage(content='I don`t know! Super far!')]}

In [7]:
add_message("hahaha, Let me show you my photo for korea", "sure. Thank you")

In [8]:
get_history()

{'history': [SystemMessage(content='The human introduces themselves as Nicolas and mentions that they live in South Korea. The AI responds by expressing excitement and finding it cool.'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!!'),
  HumanMessage(content='How far is Korea from Argentina?'),
  AIMessage(content='I don`t know! Super far!'),
  HumanMessage(content='hahaha, Let me show you my photo for korea'),
  AIMessage(content='sure. Thank you')]}

In [9]:
add_message("How about Korean view and sight", "It is so good")

In [10]:
get_history()

{'history': [SystemMessage(content='The human introduces themselves as Nicolas and mentions that they live in South Korea. The AI responds by expressing excitement and finding it cool. The human then mentions that South Korea is pretty, to which the AI responds by expressing a desire to go there.'),
  HumanMessage(content='How far is Korea from Argentina?'),
  AIMessage(content='I don`t know! Super far!'),
  HumanMessage(content='hahaha, Let me show you my photo for korea'),
  AIMessage(content='sure. Thank you'),
  HumanMessage(content='How about Korean view and sight'),
  AIMessage(content='It is so good')]}

In [1]:
from langchain.memory import ConversationKGMemory # summarize and extract most important thing
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")

In [2]:
memory.load_memory_variables({"input": "who is Nicolas"})

{'history': [SystemMessage(content='On Nicolas: Nicolas is a person. Nicolas lives in South Korea.')]}

In [3]:
add_message("Nicolas likes kimchi", "Wow that is so cool")

In [4]:
memory.load_memory_variables({"inputs": "what does nicolas like?"})

{'history': [SystemMessage(content='On Nicolas: Nicolas is a person. Nicolas lives in South Korea. Nicolas likes kimchi.')]}

In [35]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
)


chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template("{question}"),
    verbose=True, # print the logs
)

chain.predict(question="My name is Nico")



> Entering new LLMChain chain...
Prompt after formatting:
My name is Nico

> Finished chain.


'Nice to meet you, Nico! How can I assist you today?'

In [36]:
chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:
I live in Seoul

> Finished chain.


"That's great! Seoul is the capital and largest city of South Korea. It is known for its vibrant culture, modern architecture, and delicious food. There are many attractions to explore in Seoul, such as Gyeongbokgung Palace, N Seoul Tower, Myeongdong shopping district, and the Han River. The city also offers a wide range of entertainment options, including K-pop concerts, traditional performances, and trendy nightlife. Enjoy your time in Seoul!"

In [7]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=80,
    memory_key="chat_history"
)



template = """
    You are a helpful AI talking to a human.
    {chat_history}
    Human: {question}
    You:
"""


chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True, 
)

chain.predict(question="My name is Nico")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    
    
    Human: My name is Nico
    You:


> Finished chain.


'Hello Nico! How can I assist you today?'

In [8]:
chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    
    Human: My name is Nico
AI: Hello Nico! How can I assist you today?
    Human: I live in Seoul
    You:


> Finished chain.


"That's great! Seoul is a vibrant and bustling city. How can I assist you today, Nico?"

In [9]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    
    Human: My name is Nico
AI: Hello Nico! How can I assist you today?
Human: I live in Seoul
AI: That's great! Seoul is a vibrant and bustling city. How can I assist you today, Nico?
    Human: What is my name?
    You:


> Finished chain.


'Your name is Nico.'

In [10]:
chain.predict(question="My favorite korean food is bulgogi")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    
    System: The human introduces themselves as Nico.
AI: Hello Nico! How can I assist you today?
Human: I live in Seoul
AI: That's great! Seoul is a vibrant and bustling city. How can I assist you today, Nico?
Human: What is my name?
AI: Your name is Nico.
    Human: My favorite korean food is bulgogi
    You:


> Finished chain.


"Bulgogi is a delicious choice! It's a popular Korean dish made with marinated beef that is grilled to perfection. Is there anything specific you would like to know or discuss about bulgogi?"

In [39]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=80,
    memory_key="chat_history",
    return_messages=True
)


prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])


chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True, 
)

chain.predict(question="My name is Nico")




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Nico

> Finished chain.


'Hello Nico! How can I assist you today?'

In [40]:
chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Nico
AI: Hello Nico! How can I assist you today?
Human: I live in Seoul

> Finished chain.


"That's great, Nico! Seoul is a vibrant and bustling city with a rich history and culture. How can I help you with anything related to Seoul? Do you have any specific questions or need information about the city?"

In [20]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Nico
AI: Hello Nico! How can I assist you today?
Human: I live in Seoul
AI: That's great, Nico! Seoul is a vibrant and bustling city with a rich history and culture. How can I help you with anything related to Seoul?
Human: What is my name?

> Finished chain.


'Your name is Nico.'

In [21]:
chain.predict(question="My favorite korean food is bulgogi")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
System: The human introduces themselves as Nico. The AI greets Nico and asks how it can assist them.
Human: I live in Seoul
AI: That's great, Nico! Seoul is a vibrant and bustling city with a rich history and culture. How can I help you with anything related to Seoul?
Human: What is my name?
AI: Your name is Nico.
Human: My favorite korean food is bulgogi

> Finished chain.


"That's wonderful, Nico! Bulgogi is a delicious choice. It's a popular Korean dish made with thinly sliced marinated beef that is grilled or stir-fried. The beef is typically marinated in a mixture of soy sauce, sugar, garlic, sesame oil, and other seasonings, giving it a savory and slightly sweet flavor. Bulgogi is often served with rice, lettuce leaves, and various side dishes. It's definitely a must-try when visiting or living in Seoul! Is there anything else I can assist you with?"

In [17]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=80,
    memory_key="chat_history",
    return_messages=True
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}"),
])


def load_memory(input):
    print(input)
    return memory.load_memory_variables({})["chat_history"]


chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm
 # 'RunnablePassthrough' allows to call the function in chain 


def invoke_chain(question):
    result = chain.invoke({
        "question": question,
    })
    memory.save_context({"input": question}, {"output": result.content})
    print(result)


In [18]:
invoke_chain("My name is nico")

{'question': 'My name is nico'}
content='Hello Nico! How can I assist you today?'


In [19]:
invoke_chain("I like korea")

{'question': 'I like korea'}
content="That's great to hear, Nico! Korea is a fascinating country with a rich culture, history, and vibrant traditions. Is there anything specific you would like to know or discuss about Korea? I'm here to help!"


In [20]:
invoke_chain("and I like USA")

{'question': 'and I like USA'}
content="That's wonderful, Nico! The United States is a diverse and vast country with a wide range of landscapes, cultures, and opportunities. Is there anything specific you would like to know or discuss about the USA? I'm here to assist you with any questions or topics you have in mind!"


In [21]:
invoke_chain("do you know my name?")

{'question': 'do you know my name?'}
content='Yes, Nico, I know your name because you introduced yourself at the beginning of our conversation. As an AI, I have the ability to remember and recall information shared during our interaction. Is there anything else you would like to discuss or inquire about?'


In [22]:
invoke_chain("do you know what country I like?")

{'question': 'do you know what country I like?'}
content="Yes, you mentioned that you are interested in Korea. Korea is a fascinating country with a rich cultural heritage, delicious cuisine, and a vibrant entertainment industry. If you have any specific questions or topics you'd like to discuss about Korea, feel free to ask!"


In [23]:
invoke_chain("except Korea, do you know what country I like else?")

{'question': 'except Korea, do you know what country I like else?'}
content="Yes, you also mentioned that you like the United States. The United States is a diverse and vast country with a wide range of landscapes, cultures, and opportunities. If you have any specific questions or topics you'd like to discuss about the USA, I'm here to help!"


In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader

loader = TextLoader("./files/chapter_one.txt")

loader.load()

[Document(page_content="Part One \n\n\n1 \nIt was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors of Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering along with him. \n\nThe hallway smelt of boiled cabbage and old rag mats. At one end of it a coloured poster, too large for indoor display, had been tacked to the wall. It depicted simply an enormous face, more than a metre wide: the face of a man of about forty-five, with a heavy black moustache and ruggedly handsome features. Winston made for the stairs. It was no use trying the lift. Even at the best of times it was seldom working, and at present the electric current was cut off during daylight hours. It was part of the economy drive in preparation for Hate Week. The flat was seven flights up, and Winston, who was thirty-nine and had a varicose ulcer 

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader

loader = TextLoader("./files/chapter_one.txt")

len(loader.load()) # 1

1

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter() # split the file`s content by paragraph

loader = TextLoader("./files/chapter_one.txt")

docs = loader.load()
splitter.split_documents(docs)

[Document(page_content="Part One \n\n\n1 \nIt was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors of Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering along with him. \n\nThe hallway smelt of boiled cabbage and old rag mats. At one end of it a coloured poster, too large for indoor display, had been tacked to the wall. It depicted simply an enormous face, more than a metre wide: the face of a man of about forty-five, with a heavy black moustache and ruggedly handsome features. Winston made for the stairs. It was no use trying the lift. Even at the best of times it was seldom working, and at present the electric current was cut off during daylight hours. It was part of the economy drive in preparation for Hate Week. The flat was seven flights up, and Winston, who was thirty-nine and had a varicose ulcer 

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter() 

loader = TextLoader("./files/chapter_one.txt")


loader.load_and_split(text_splitter=splitter)


[Document(page_content="Part One \n\n\n1 \nIt was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors of Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering along with him. \n\nThe hallway smelt of boiled cabbage and old rag mats. At one end of it a coloured poster, too large for indoor display, had been tacked to the wall. It depicted simply an enormous face, more than a metre wide: the face of a man of about forty-five, with a heavy black moustache and ruggedly handsome features. Winston made for the stairs. It was no use trying the lift. Even at the best of times it was seldom working, and at present the electric current was cut off during daylight hours. It was part of the economy drive in preparation for Hate Week. The flat was seven flights up, and Winston, who was thirty-nine and had a varicose ulcer 

In [32]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter() 

loader = TextLoader("./files/chapter_one.txt")


docs = loader.load()

splitResult = splitter.split_documents(docs)

len(splitResult) # 11

11

In [33]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter() 

loader = TextLoader("./files/chapter_one.txt")


splitResult = loader.load_and_split(text_splitter=splitter)

len(splitResult) # 11

11

In [37]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
) 

loader = TextLoader("./files/chapter_one.txt")


splitResult = loader.load_and_split(text_splitter=splitter)

len(splitResult) # 3683


3683

In [36]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50,
) 

loader = TextLoader("./files/chapter_one.txt")


splitResult = loader.load_and_split(text_splitter=splitter)

len(splitResult) # 248

248

In [35]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
) 

loader = TextLoader("./files/chapter_one.txt")


splitResult = loader.load_and_split(text_splitter=splitter)

len(splitResult) # 39

Created a chunk of size 964, which is longer than the specified 600
Created a chunk of size 775, which is longer than the specified 600
Created a chunk of size 955, which is longer than the specified 600
Created a chunk of size 923, which is longer than the specified 600
Created a chunk of size 1169, which is longer than the specified 600
Created a chunk of size 822, which is longer than the specified 600
Created a chunk of size 701, which is longer than the specified 600
Created a chunk of size 746, which is longer than the specified 600
Created a chunk of size 736, which is longer than the specified 600
Created a chunk of size 1111, which is longer than the specified 600
Created a chunk of size 992, which is longer than the specified 600
Created a chunk of size 991, which is longer than the specified 600
Created a chunk of size 1742, which is longer than the specified 600
Created a chunk of size 2002, which is longer than the specified 600
Created a chunk of size 1901, which is longe

39

In [39]:
from langchain.embeddings import OpenAIEmbeddings

embedder = OpenAIEmbeddings()

vector = embedder.embed_query("Hi")

vector


[-0.03633027945867064,
 -0.007150246393661463,
 -0.03344155148607811,
 -0.028835205754777583,
 -0.026805291318151986,
 0.03435241105351516,
 -0.012270577419374741,
 -0.007904958087801004,
 0.0019209374702995232,
 -0.0029895706510255785,
 0.024619229473890177,
 -0.0024007652282535437,
 -0.005657087623047437,
 -0.0029830646642611675,
 0.006694816609942938,
 -0.0029245093863975876,
 0.03404011623824273,
 -0.0014703873512772927,
 0.020936753886571638,
 -0.009297272317336807,
 -0.02145724586624074,
 0.010370784580682538,
 0.006262158594807117,
 0.007319405774826497,
 -0.012251058760589567,
 0.0008645031602758928,
 0.005634315854464735,
 -0.00995439211453436,
 -0.0030432462641928654,
 -0.024528142399559368,
 0.0105984994725418,
 -0.013884099389852161,
 -0.024476092642798902,
 -0.01401422285043073,
 0.002418657099309306,
 -0.018880814571565808,
 0.0003157510407054849,
 -0.011366224071532751,
 0.01786585735325301,
 -0.009941379675344246,
 0.013259510690629896,
 -0.01113200296007843,
 -0.009323

In [13]:
from langchain.embeddings import OpenAIEmbeddings

embedder = OpenAIEmbeddings()

vector = embedder.embed_query("Hi")

len(vector) # 1536

1536

In [40]:
from langchain.embeddings import OpenAIEmbeddings

embedder = OpenAIEmbeddings()

vector = embedder.embed_documents([
    "hi",
    "how",
    "are",
    "you longer sentences because"
])

vector

[[-0.03503197280802852,
  -0.020601634912212417,
  -0.015375726194590843,
  -0.03986396841512036,
  -0.02741632492572226,
  0.021126852392918723,
  -0.022072243485661065,
  -0.01945928772986648,
  -0.00959177460221072,
  -0.013143554695556638,
  0.029569713988915027,
  -0.00471710486526392,
  -0.015178770104987246,
  -0.014115205358482746,
  0.009053427336412527,
  0.015113118385560224,
  0.03831457554318521,
  -0.0058003647550675125,
  0.023831720369646633,
  -0.012749641585026906,
  -0.014916161364634091,
  -0.0030216387103995197,
  -0.006919733416518854,
  -0.008357515198931458,
  -0.022715632757484464,
  -0.00016864388455371465,
  0.01356372830759267,
  -0.016977637275571,
  0.004500452701038695,
  -0.022321720578277266,
  0.014601031621268334,
  -0.0008920481975930099,
  -0.04495857089992029,
  -0.009670557038052158,
  -0.009788730691814318,
  -0.015677727084187253,
  0.009828121909735038,
  -0.021179373395931327,
  0.015060596451225086,
  -0.005537756480375626,
  0.00818681961083

In [42]:
from langchain.embeddings import OpenAIEmbeddings

embedder = OpenAIEmbeddings()

vector = embedder.embed_documents([
    "hi",
    "how",
    "are",
    "you longer sentences because",
])

len(vector) # 4

4

In [43]:
from langchain.embeddings import OpenAIEmbeddings

embedder = OpenAIEmbeddings()

vector = embedder.embed_documents([
    "hi",
    "how",
    "are",
    "you longer sentences because"
])

len(vector[0]) # 1536

1536

In [45]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
) 

loader = TextLoader("./files/chapter_one.txt")


docs = loader.load_and_split(text_splitter=splitter)

embedder = OpenAIEmbeddings()

vectorstore = Chroma.from_documents(docs, embedder)



In [ ]:
vectorstore.similarity_search("where does winston live?")

In [46]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
) 

loader = TextLoader("./files/chapter_one.txt")


docs = loader.load_and_split(text_splitter=splitter)

embedder = OpenAIEmbeddings()



cache_dir = LocalFileStore("./.cache/")

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embedder, cache_dir)



vectorstore = Chroma.from_documents(docs, cached_embeddings)

In [ ]:
vectorstore.similarity_search("where does winston live?")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)



splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
) 

loader = TextLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)


vectorstore = Chroma.from_documents(docs, cached_embeddings)



chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
)

chain.run("Where does Winston live?") 

# 'Winston Smith lives in Victory Mansions.'



In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)



splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
) 

loader = TextLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)


vectorstore = FAISS.from_documents(docs, cached_embeddings)


chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",
    retriever=vectorstore.as_retriever(),
)

chain.run("Where does Winston live?") 

# 'In the provided context, it is not explicitly mentioned where Winston lives. However, it can be inferred that Winston lives in an apartment, possibly located in the vicinity of the Ministry of Love. The description suggests that Winston has a tiny kitchen and a living room with a small table situated to the left of the telescreen.'


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)



splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
) 

loader = TextLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)


vectorstore = FAISS.from_documents(docs, cached_embeddings)


chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_reduce",
    retriever=vectorstore.as_retriever(),
)

chain.run("Where does Winston live?") 

# 'There is no information provided about where Winston lives.'



In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)



splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
) 

loader = TextLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)


vectorstore = FAISS.from_documents(docs, cached_embeddings)


chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_rerank",
    retriever=vectorstore.as_retriever(),
)

chain.run("Where does Winston live?") 

# 'Winston lives in Victory Mansions.'



In [24]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature=0.1
)

cache_dir = LocalFileStore("./.cache/")

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)



splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
) 

loader = TextLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)


vectorstore = FAISS.from_documents(docs, cached_embeddings)


retriever = vectorstore.as_retriever()


prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}"),
    ("human", "{question}")
])



chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm
# the invoke()`s input can be passed to ' RunnablePassthrough()'



chain.invoke("Where does Winston live?") 

# AIMessage(content='Winston Smith lives in Victory Mansions.')



--- Logging error ---
Traceback (most recent call last):
  File "/Users/gimseongjin/FullStack_GPT/env/lib/python3.11/site-packages/langchain/callbacks/manager.py", line 2072, in _configure
    handler = LangChainTracer(project_name=tracer_project)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/gimseongjin/FullStack_GPT/env/lib/python3.11/site-packages/langchain/callbacks/tracers/langchain.py", line 97, in __init__
    self.client = client or get_client()
                            ^^^^^^^^^^^^
  File "/Users/gimseongjin/FullStack_GPT/env/lib/python3.11/site-packages/langchain/callbacks/tracers/langchain.py", line 55, in get_client
    _CLIENT = Client()
              ^^^^^^^^
  File "/Users/gimseongjin/FullStack_GPT/env/lib/python3.11/site-packages/langsmith/client.py", line 272, in __init__
    _validate_api_key_if_hosted(self.api_url, self.api_key)
  File "/Users/gimseongjin/FullStack_GPT/env/lib/python3.11/site-packages/langsmith/client.py", line 181, in 

AIMessage(content='Winston Smith lives in Victory Mansions.')

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(
    temperature=0.1
)

cache_dir = LocalFileStore("./.cache/")

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)



splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
) 

loader = TextLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)


vectorstore = FAISS.from_documents(docs, cached_embeddings)


retriever = vectorstore.as_retriever()




map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim. If there is no relevant text, return : ''
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

map_doc_chain = map_doc_prompt | llm



def map_docs(inputs):
    # print(inputs)
    documents = inputs['documents']
    question = inputs['question']

    return "\n\n".join(
        map_doc_chain.invoke(
            {"context": doc.page_content, "question": question}).content
            for doc in documents
        )
    


map_chain = { "documents": retriever, "question": RunnablePassthrough() } | RunnableLambda(map_docs) # can call a function in anywhere via 'RunnableLambda()'


final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

chain = {"context": map_chain, "question": RunnablePassthrough()} | final_prompt | llm


chain.invoke("Where does Winston live?")  # AIMessage(content='Winston lives in Victory Mansions.')


chain.invoke("Where does Winston go to work?") # AIMessage(content='Winston goes to work at the Ministry of Truth.')
